<a href="https://colab.research.google.com/github/niranjana2222/Classifying-Natural-and-Synthetic-Products/blob/main/cheminformatics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports
import pandas as pd
import math
import zipfile
import os
import numpy as np

In [ ]:
#create dataframes
all_nat_csv = []
count = 0
for i in os.listdir('/content/nat_csv'):
    if count > 12:
        break
    all_nat_csv.append(pd.read_csv(f'/content/nat_csv/{i}', lineterminator='\n', encoding='latin1', on_bad_lines='skip'))
    count +=1

nat_df = pd.concat(all_nat_csv)
 
all_syn_csv= []
count = 0
for i in os.listdir('/content/syn_csv'):
    if count > 12:
        break
    all_syn_csv.append(pd.read_csv(f'/content/syn_csv/{i}', lineterminator='\n', encoding='latin1', on_bad_lines='skip'))
    count += 1

syn_df = pd.concat(all_syn_csv)
#syn_df.describe()
 
#create labels
nat_df['label'] = 0 #natural = 0
syn_df['label'] = 1 #synthetic = 1

#create final dataframe
final_df = pd.concat([nat_df, syn_df])
del final_df['smiles']

#replace bad data
nan_value=False
for i in final_df.columns:
    
    for j in final_df[i]:
        if (math.isnan(j)):
            nan_value=True
        
        if (j >= 1.7976931348623157e+308):
            final_df[i].replace(j, final_df[i].mean())
            
    if nan_value:
        nan_value=False
        final_df.fillna(value=final_df[i].mean(), inplace=True)
        
        
final_df.replace(np.inf, float(0), inplace=True )
final_df.replace(-np.inf, float(0), inplace=True )


In [ ]:
from sklearn.model_selection import train_test_split

#create training and test data
y = final_df['label']
del final_df['label']
X = final_df
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values)

In [ ]:

#train decision tree classifier
from sklearn import tree
decision_tree = tree.DecisionTreeClassifier(max_depth=16)
decision_tree.fit(X_train, y_train)

#train extra trees classifier
from sklearn.ensemble import ExtraTreesClassifier
extra_trees = ExtraTreesClassifier(max_depth=16)
extra_trees.fit(X_train,y_train)

#train random forest classifier
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(max_depth=16)
random_forest.fit(X_train,y_train)


NameError: ignored

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

def evaluate(tree):
  y_pred = tree.predict(X_test)
  print("Accuracy: ", accuracy_score(y_pred, y_test))
  plot_confusion_matrix(tree, X_test, y_test)  
  plt.show()
  
  print('\n', classification_report(y_test,y_pred))

  feat_importances = pd.Series(tree.feature_importances_, index=final_df.columns)
  feat_importances.nlargest(10).plot(kind='barh')
  plt.show()

print("Decision Tree")
evaluate(decision_tree)
print("Extra Trees")
evaluate(extra_trees)
print("Random Forest")
evaluate(random_forest)

print("0 = Natural")
print("1 = Synthetic")

In [ ]:
importance = decision_tree.feature_importances_
j = 0
indices = []

# remove unused features with importance of 0
for j,v in enumerate(importance):
    if v <= 0:
      indices.append(j)
    j+=1

#add feature index
feature_numbers = np.arange(1876)
feature_numbers = np.delete(feature_numbers, indices)
actual_importances = np.delete(importance, indices)
used_features = np.column_stack((feature_numbers, actual_importances))

#sort features in reverse
sorted_features = np.sort(used_features, axis=0)[::-1]

j = 0
indices = []
for k, (i, v) in enumerate(sorted_features):
  if j < 10:
    print('Feature: %0d, Name: %s Score: %.5f' % (i, final_df.columns[i], v))
  else:
    indices.append(j)
  j+=1

top_ten = np.delete(sorted_features, indices)

Feature: 1456, Name: TDB2m Score: 0.30757
Feature: 1443, Name: XLogP Score: 0.17040
Feature: 1440, Name: WTPT-5 Score: 0.11240
Feature: 1437, Name: WTPT-2 Score: 0.08011
Feature: 1434, Name: MW Score: 0.05166
Feature: 1315, Name: nFRing Score: 0.04533
Feature: 1275, Name: MLFER_BH Score: 0.03533
Feature: 1267, Name: MDEO-22 Score: 0.02631
Feature: 1175, Name: ETA_BetaP_s Score: 0.02573
Feature: 1159, Name: ETA_Epsilon_3 Score: 0.02405


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: Indexing with a float is deprecated, and will raise an IndexError in pandas 2.0. You can manually convert to an integer key instead.
